In [13]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# auto-reload library if developing library functionalities
%reload_ext autoreload
%autoreload 2
# Add the old-can-decoder-c56d4fe6 directory to sys.path
sys.path.append(os.path.abspath('old-can-decoder-c56d4fe6'))

# import library
from PandaCANDecoder.decoder import Decoder as OldDecoder
from can_decoder.decoder import Decoder

In [14]:
# Import df and divide by networks

df_all = pd.read_csv('data/SampleRoadAcceleration_ALL.csv')

# decoder_FD3 = Decoder(
#     df_all[df_all['network'] == 'DW CAN 01'].drop(columns=['network']).reset_index(drop=True)
#     )

# decoder_FD2 = Decoder(
#     df_all[df_all['network'] == 'DW CAN 07'].drop(columns=['network']).reset_index(drop=True)
#     )

# decoder_FD11 = Decoder(
#     df_all[df_all['network'] == 'DW CAN 02'].drop(columns=['network']).reset_index(drop=True)
#     )

# decoder_IHS = Decoder(
#     df_all[df_all['network'] == 'DW CAN 05'].drop(columns=['network']).reset_index(drop=True)
#     )

# decoder_IHSAT = Decoder(
#     df_all[df_all['network'] == 'DW CAN 06'].drop(columns=['network']).reset_index(drop=True)
#     )

# decoder_C = Decoder(
#     df_all[df_all['network'] == 'DW CAN 08'].drop(columns=['network']).reset_index(drop=True)
#     )

# NETWORK: CAN FD3

## Initial

In [16]:
decoder_FD3 = Decoder(
    df_all[df_all['network'] == 'DW CAN 01'].drop(columns=['network']).reset_index(drop=True)
    )

In [17]:
## Parameters, Load Messages, Dynamic Filtering, 
byte_filters = {
    "18DA10F1": ["1", "2", "3", "4",'5'],
    "18DAF110": ["1", "2", "3", "4",'5'],
    "18DAF218": ["1", "2", "3", "4",'5'],
    "18DA18F2": ["1", "2", "3", "4",'5'],
}

decoder_FD3.generate_msgs(byte_filters=byte_filters)

## Dynamic filtering: if byte 1 is 0 we should discard byte 5
for msg in decoder_FD3.msgs:
    if msg.msg_byte_filter is not None:
        if msg.msg_byte_filter['1'] != 0:
            msg.msg_byte_filter.pop('5', None)
            # Remove duplicates

# Remove duplicate messages with same msg_id and identical msg_byte_filter after discarding byte 5
unique_msgs = []
seen = set()
for msg in decoder_FD3.msgs:
    key = (msg.msg_id, tuple(sorted(msg.msg_byte_filter.items())) if msg.msg_byte_filter else None)
    if key not in seen:
        unique_msgs.append(msg)
        seen.add(key)
decoder_FD3.msgs = unique_msgs



Generated 910 messages


In [18]:
## TS data and signal decoding

decoder_FD3.generate_msg_ts_data(rewrite=True)

decoder_FD3.calculate_signals(
        tokenization_method='conditional_bit_flip',
        signedness_method='msb_classifier',
        alpha1=0.01,
        alpha2=0.5, #0.5,
        gamma1=0.2)

## Signal Setup

In [ ]:
## Aux Functions
from can_decoder.message import Message
from can_decoder.signal import Signal


def update_message(decoder, msg_id, diag_id,signals):
    msgs = decoder.msgs
    filtered_msgs = [
        msg for msg in msgs
        if msg.msg_id == msg_id and msg.msg_byte_filter is not None and
        list(msg.msg_byte_filter.values())[-2:] == diag_id
    ]
    if len(filtered_msgs) != 1:
        raise ValueError("Multiple or no messages found for given msg_id and diag_id")
    m = filtered_msgs[0]
    m.signals = [
        Signal(
            start_bit=signal[0],
            length=signal[1],
            byte_order=signal[2],
            classification=signal[3],
            msg=m,
            name=signal[4],
        ) for signal in signals
    ]

# # Auxiliary view of messages filters
# # [m for m in decoder_FD3.msgs if m.msg_id in ['18DAF218', '18DAF110', '18DAF228']]
# sorted([
#     (f"{m.msg_id}", 
#      [hex(int(f)).rjust(4) for f in m.msg_byte_filter.values()])
#     for m in decoder_FD3.msgs if m.msg_id in ['18DAF218']
# ], key=lambda el: int(el[1][-2], 16)*16 + int(el[1][-1], 16))

# # Aux
# decoder_FD3.plot_message_from_id('18DAF218',[0x6,0x62,0x5,0x45])

In [106]:
# # Auxiliary view of messages filters
# # [m for m in decoder_FD3.msgs if m.msg_id in ['18DAF218', '18DAF110', '18DAF228']]
# sorted([
#     (f"{m.msg_id}", 
#      [hex(int(f)).rjust(4) for f in m.msg_byte_filter.values()])
#     for m in decoder_FD3.msgs if m.msg_id in ['18DAF110']
# ], key=lambda el: int(el[1][-2], 16)*16 + int(el[1][-1], 16))

In [ ]:
# PCM - 18DAF110

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x1, 0x06],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_Accum_Air_Port_Mass_Flow__G'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x1, 0xC9],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_Purge_Air_Flow__g_per_s'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x1, 0xCA],
               signals = [
                   (39, 32, 'be', 'ts', 'PCM_Mass_Air_Flow__g_per_s'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x1, 0xD5],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_Engine_Speed_RPM'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x2, 0x27],
               signals = [
                   (39, 8, 'be', 'ts', 'PCM_Fuel_Level__%'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x2, 0x2A],
               signals = [
                   (39, 8, 'be', 'ts', 'PCM_Oil_Pressure__kPa'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x2, 0x33],
               signals = [
                   (39, 32, 'be', 'ts', 'PCM_Avg_Total_Working_Power__us'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x2, 0x34],
               signals = [
                   (39, 32, 'be', 'ts', 'PCM_Injector_Pulse_Width_Cyl1__us'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x2, 0x35],
               signals = [
                   (39, 32, 'be', 'ts', 'PCM_Injector_Pulse_Width_Cyl2__us'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x2, 0x36],
               signals = [
                   (39, 32, 'be', 'ts', 'PCM_Injector_Pulse_Width_Cyl3__us'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x2, 0x37],
               signals = [
                   (39, 32, 'be', 'ts', 'PCM_Injector_Pulse_Width_Cyl4__us'),
               ])


update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x2, 0x55],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_Fuel_Mass_Cyl1__mg_per_charge'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x2, 0x56],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_Fuel_Mass_Cyl2__mg_per_charge'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x2, 0x57],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_Fuel_Mass_Cyl3__mg_per_charge'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x2, 0x58],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_Fuel_Mass_Cyl4__mg_per_charge'),
               ])


update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x2, 0xB2],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_Req_Fuel_Charge_Cyl1__mg_per_charge'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x2, 0xB3],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_Req_Fuel_Charge_Cyl2__mg_per_charge'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x2, 0xB4],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_Req_Fuel_Charge_Cyl3__mg_per_charge'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x2, 0xB5],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_Req_Fuel_Charge_Cyl4__mg_per_charge'),
               ])



update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x3, 0xE1],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_MisFire_Counter_Cat_200rev__cnt'),
               ])


update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x3, 0xE3],
               signals = [
                   (39, 8, 'be', 'ts', 'PCM_MisFire_Counter_600rev__cnt'),
               ])


update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x3, 0xE4],
               signals = [
                   (39, 8, 'be', 'ts', 'PCM_MisFire_Counter_4000rev__cnt'),
               ])



update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x3, 0xF0],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_RPM_Engine_Misfiring__rpm'),
               ])



update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x6, 0x7C],
               signals = [
                   (39, 8, 'be', 'ts', 'PCM_Spark_Advance__eng_deg'),
               ])



update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x6, 0xDA],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_Actual_Torque__Nm'),
               ])



update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x12, 0x47],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_Throttle_Inlet_Pressure__kPa'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF110', diag_id = [0x15, 0x43],
               signals = [
                   (39, 16, 'be', 'ts', 'PCM_MakeUp_Air_Pressure__kPa'),
               ])



In [88]:
# TCM - 18DAF218

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF218', diag_id = [0x5, 0x00],
               signals = [
                   (39, 16, 'be', 'ts', 'TCM_Torque_Converter_Slip__Nm'),
               ])



update_message(decoder = decoder_FD3, 
               msg_id = '18DAF218', diag_id = [0x5, 0x1F],
               signals = [
                   (71, 16, 'be', 'ts', 'TCM_Engine_Speed__rpm'),
                   (87, 24, 'be', 'ts', 'TCM_Transm_Output_Shaft_Speed__rpm'),
                   (111, 8, 'be', 'ts', 'TCM_Oil_Temp__C'),
               ])


update_message(decoder = decoder_FD3, 
               msg_id = '18DAF218', diag_id = [0x10, 0x18],
               signals = [
                   (39, 16, 'be', 'ts', 'TCM_Crankshaft_Torque_Actual__Nm'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF218', diag_id = [0x50, 0x10],
               signals = [
                   (39, 16, 'be', 'ts', 'TCM_Lateral_Accel__mps2'),
                   (55, 16, 'be', 'ts', 'TCM_Longit_Accel__mps2'),
               ])



update_message(decoder = decoder_FD3, 
               msg_id = '18DAF218', diag_id = [0x50, 0x20],
               signals = [
                   (47, 16, 'be', 'ts', 'TCM_Brake_Torque__Nm'),
                   (63, 16, 'be', 'ts', 'TCM_Brake_Driver_Torque__Nm'),
                   (79, 16, 'be', 'ts', 'TCM_Brake_Torque__Nm'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF218', diag_id = [0x50, 0x21],
               signals = [
                   (55, 16, 'be', 'ts', 'TCM_Brake_Boost_Pressure__bar'),
                   (79, 16, 'be', 'ts', 'TCM_Brake_Pedal_Pressure__bar'),
                   (95, 16, 'be', 'ts', 'TCM_Brake_Total_Friction_Torque_Commanded__Nm'),
                   (135, 16, 'be', 'ts', 'TCM_Regen_Braking_Torque_Request__Nm'),
                   (167, 16, 'be', 'ts', 'TCM_Torque_Request__Nm'),
                   (183, 16, 'be', 'ts', 'TCM_Total_Brake_Torque__Nm'),
                   (199, 16, 'be', 'ts', 'TCM_Vehicle_Speed_VSO__kph'),

               ])


update_message(decoder = decoder_FD3, 
               msg_id = '18DAF218', diag_id = [0x50, 0x22],
               signals = [
                   (55, 16, 'be', 'ts', 'TCM_WheelSpeed_LF__kph'),
                   (79, 16, 'be', 'ts', 'TCM_WheelSpeed_LR__kph'),
                   (103, 16, 'be', 'ts', 'TCM_WheelSpeed_RF__kph'),
                   (127, 16, 'be', 'ts', 'TCM_WheelSpeed_RR__kph'),
               ])



update_message(decoder = decoder_FD3, 
               msg_id = '18DAF218', diag_id = [0x50, 0x25],
               signals = [
                   (39, 8, 'be', 'ts', 'TCM_Brake_Pedal_Position__%'),
                   (40, 1, 'be', 'ts', 'TCM_Brake_Pedal_Status__state'),
               ])




update_message(decoder = decoder_FD3, 
               msg_id = '18DAF218', diag_id = [0x50, 0x30],
               signals = [
                   (111, 8, 'be', 'ts', 'TCM_Engine_Status__state'),
                   (143, 8, 'be', 'ts', 'TCM_Accel_Pedal_Position__%'),
                   (159, 8, 'be', 'ts', 'TCM_Intake_Air_Temperature__C'),
                   (176, 1, 'be', 'ts', 'TCM_Powertrain_Propulsion_Status__state'),
                   (191, 8, 'be', 'ts', 'TCM_Transmission_Clutch_Slip_Mode__state'),
                   (192, 1, 'be', 'ts', 'TCM_Transmission_AC_Converter_Clutch_Slipping__state'),
               ])


update_message(decoder = decoder_FD3, 
               msg_id = '18DAF218', diag_id = [0x50, 0x31],
               signals = [
                   (55, 8, 'be', 'ts', 'TCM_Engine_Oil_Temp__C'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF218', diag_id = [0x50, 0x33],
               signals = [
                   (39, 8, 'be', 'ts', 'TCM_Engine_Water_Temp__C'),
                   (47, 8, 'be', 'ts', 'TCM_Terrain_Mode__state'),
               ])




update_message(decoder = decoder_FD3, 
               msg_id = '18DAF218', diag_id = [0x50, 0x3A],
               signals = [
                   (47, 16, 'be', 'ts', 'TCM_Unknown__'),
                   (63, 16, 'be', 'ts', 'TCM_Sum_Torque_Driver_Request_Mode__Nm%'),
                   (79, 16, 'be', 'ts', 'TCM_Sum_Torque_Max__Nm'),
                   (95, 16, 'be', 'ts', 'TCM_Sum_Torque_Min__Nm'),
                   (111, 16, 'be', 'ts', 'TCM_Corrected_Driver_Torque_Demand__Nm'),
                   (127, 16, 'be', 'ts', 'TCM_Sum_Torque_Static__Nm'),
                   (143, 16, 'be', 'ts', 'TCM_StaticSum_Torque_Transmission_Torque_Request__Nm'),
                   (159, 16, 'be', 'ts', 'TCM_Target_Torque_Transmission_Torque_Request__Nm'),
               ])


update_message(decoder = decoder_FD3, 
               msg_id = '18DAF218', diag_id = [0x50, 0x40],
               signals = [
                   (47, 16, 'be', 'ts', 'TCM_Eletrical_Machine_Torque_Available_Max__Nm'),
                   (63, 16, 'be', 'ts', 'TCM_Eletrical_Machine_Torque_Available_Min__Nm'),
                   (79, 16, 'be', 'ts', 'TCM_Eletrical_Machine_Current_Torque__Nm'),
                   (95, 16, 'be', 'ts', 'TCM_Engine_P1_Torque__Nm'),
                   (111, 16, 'be', 'ts', 'TCM_Est_Regen_Braking_FlyWheel_Torque__Nm'),
                   (127, 16, 'be', 'ts', 'TCM_HCP_Electric_Motor_Target_Torque__Nm'),
                   (139, 4, 'be', 'ts', 'TCM_HCP_Request_Gear__num'),
               ])

update_message(decoder = decoder_FD3, 
               msg_id = '18DAF218', diag_id = [0x5, 0x45],
               signals = [
                   (39, 8, 'be', 'ts', 'TCM_AWD_System_Status__state'),
                   (42, 3, 'be', 'ts', 'TCM_Terrain_Mode_Status__state'),
               ])
try:
    update_message(decoder = decoder_FD3, 
                msg_id = '18DAF218', diag_id = [0x5, 0x1A],
                signals = [
                    (39, 4, 'be', 'ts', 'TCM_Target_Gear__state'),
                    (35, 4, 'be', 'ts', 'TCM_Current_Gear__state'),
                ])
except Exception as e:
    pass



In [136]:
reference_messages = [
    m for m in decoder_FD3.msgs if m.msg_id in ['18DAF218', '18DAF110', '18DAF228']
]

hs_messages =[
    m for m in decoder_FD3.msgs if len(m.msg_id) <= 3
]

In [147]:
test_hs_messages = [
    decoder_FD3.get_message('40B'),
    decoder_FD3.get_message('114'),
    decoder_FD3.get_message('10D')
    ]
for m in test_hs_messages:
    for s in m.signals:
        print(s if s.classification == 'ts' else None)

test_reference_messages = [
    decoder_FD3.get_message('18DAF110', [5, 98, 1, 213]),
    decoder_FD3.get_message('18DAF218', [0,25, 98, 0x50, 0x40]),
    decoder_FD3.get_message('18DAF110', [5, 98, 6, 218])
    ]


Msg: 40B        Signal: S_40B_BE_0           Type: ts    Sign: unsigned   Bit: 7     Length: 16   
None
Msg: 40B        Signal: S_40B_BE_2           Type: ts    Sign: unsigned   Bit: 134   Length: 1    
Msg: 40B        Signal: S_40B_BE_3           Type: ts    Sign: unsigned   Bit: 133   Length: 6    
None
Msg: 40B        Signal: S_40B_BE_5           Type: ts    Sign: unsigned   Bit: 150   Length: 7    
None
Msg: 40B        Signal: S_40B_BE_7           Type: ts    Sign: unsigned   Bit: 164   Length: 5    
None
Msg: 40B        Signal: S_40B_BE_9           Type: ts    Sign: unsigned   Bit: 179   Length: 12   
None
None
Msg: 40B        Signal: S_40B_LE_1           Type: ts    Sign: unsigned   Bit: 184   Length: 8    
None
Msg: 40B        Signal: S_40B_LE_3           Type: ts    Sign: unsigned   Bit: 176   Length: 4    
None
Msg: 40B        Signal: S_40B_LE_5           Type: ts    Sign: unsigned   Bit: 160   Length: 5    
None
Msg: 40B        Signal: S_40B_LE_7           Type: ts    Sign: u

## Full Network Diagnostic Signal Matching

In [148]:
def match_table(reference_messages, hs_messages):
    match_table = [] 
    for i, m in enumerate(reference_messages):
        print("----------------------\n"+f"{i+1}/{len(reference_messages):<10} Message:   {m.msg_id:<8} | {len(m.signals)} signals")
        for j, s in enumerate(m.signals):
            print("\n")
            print(f"{j+1}/{len(m.signals):<10} Signal:   {s.msg.msg_id:<8} | {s.name}")

            try:
                candidates = decoder_FD3.find_signal_match(
                    s,
                    thresh=0.8,
                    only_ts=True,
                    messages=hs_messages
                )
            except Exception as e:
                print(f"Error finding signal match: {e}")
                candidates = []

            if len(candidates) > 0:
                match_table.append((m, s, candidates))
            else:
                match_table.append((m, s, None))
    return match_table

test_table = match_table(test_reference_messages, test_hs_messages)

----------------------
1/3          Message:   18DAF110 | 1 signals
1/1          Signal:   18DAF110 | PCM_Engine_Speed_RPM
PCM_Engine_Speed_RPM --> S_40B_BE_0           in 40B        , r^2=0.999951
PCM_Engine_Speed_RPM --> S_40B_LE_13          in 40B        , r^2=0.999924
PCM_Engine_Speed_RPM --> S_114_BE_5           in 114        , r^2=0.999512
PCM_Engine_Speed_RPM --> S_114_LE_54          in 114        , r^2=0.996725
----------------------
2/3          Message:   18DAF218 | 7 signals
2/7          Signal:   18DAF218 | TCM_Eletrical_Machine_Torque_Available_Max__Nm
TCM_Eletrical_Machine_Torque_Available_Max__Nm --> S_10D_BE_13          in 10D        , r^2=0.999712
TCM_Eletrical_Machine_Torque_Available_Max__Nm --> S_10D_LE_26          in 10D        , r^2=0.802328
2/7          Signal:   18DAF218 | TCM_Eletrical_Machine_Torque_Available_Min__Nm
TCM_Eletrical_Machine_Torque_Available_Min__Nm --> S_10D_BE_15          in 10D        , r^2=0.999505
TCM_Eletrical_Machine_Torque_Available_Min__

In [149]:
test_table

[(Message 18DAF110 - Filter [B1:5|B2:98|B3:1|B4:213] : Length 8 bytes,
  Msg: 18DAF110   Signal: PCM_Engine_Speed_RPM Type: ts    Sign: unsigned   Bit: 39    Length: 16   ,
  [(Msg: 40B        Signal: S_40B_BE_0           Type: ts    Sign: unsigned   Bit: 7     Length: 16   ,
    LinregressResult(slope=np.float64(7.9956217614539185), intercept=np.float64(4.326958552772339), rvalue=np.float64(0.999975343347519), pvalue=np.float64(2.1638341758723607e-303), stderr=np.float64(0.004745459919644161), intercept_stderr=np.float64(9.1340542157407)),
    np.float64(0.9999506873029885)),
   (Msg: 40B        Signal: S_40B_LE_13          Type: ts    Sign: unsigned   Bit: 0     Length: 8    ,
    LinregressResult(slope=np.float64(0.031090343569652006), intercept=np.float64(-0.07288146522715522), rvalue=np.float64(0.9999620477907021), pvalue=np.float64(2.794048549372925e-290), stderr=np.float64(2.2893247032570705e-05), intercept_stderr=np.float64(0.04406488793725303)),
    np.float64(0.99992409702177

In [180]:
# One plot per message

# Group signals by unique message using a regular dict
message_signals = {}
for msg, signal, candidates in test_table:
    if msg not in message_signals:
        message_signals[msg] = []
        signal_dict = {'signal': signal, 'candidates': candidates}
    message_signals[msg].append(signal_dict)

for msg, signals in message_signals.items():
    to_plot = []
    for signal in signals:
        to_plot.append(signal['signal'])
        to_plot.append(signal['candidates'][0][0] if signal['candidates'] and len(signal['candidates']) > 0 else None)
        to_plot.append(signal['candidates'][1][0] if signal['candidates'] and len(signal['candidates']) > 1 else None)
    print(f"Plotting {msg.msg_id} with {len(signals)} signals")
    f = decoder_FD3.plot_signals(to_plot,return_fig=True)
    f.show()


Plotting 18DAF110 with 1 signals


Plotting 18DAF218 with 7 signals


Plotting 18DAF110 with 1 signals


AttributeError: 'NoneType' object has no attribute 'ts_data_timestamps'

In [179]:
list(message_signals.items())[0][1][0]['candidates'][0][0]

Msg: 40B        Signal: S_40B_BE_0           Type: ts    Sign: unsigned   Bit: 7     Length: 16   

In [112]:
candidates = decoder_FD3.find_signal_match(
    decoder_FD3.get_signal('18DAF110', [5, 98, 1, 213], 'PCM_Engine_Speed_RPM'),
    thresh=0.9,
    only_ts=True,
    messages = [
        decoder_FD3.get_message('106'),
        decoder_FD3.get_message('114'),
    ]
)
f = decoder_FD3.plot_signal_matches(
    decoder_FD3.get_signal('18DAF110', [5, 98, 1, 213], 'PCM_Engine_Speed_RPM'),
    candidates=candidates,
    return_fig=True)
f.show()

PCM_Engine_Speed_RPM --> S_106_BE_1           in 106        , r^2=0.993198
PCM_Engine_Speed_RPM --> S_106_BE_3           in 106        , r^2=0.999987
PCM_Engine_Speed_RPM --> S_106_LE_31          in 106        , r^2=0.997794
PCM_Engine_Speed_RPM --> S_106_LE_34          in 106        , r^2=0.990037
PCM_Engine_Speed_RPM --> S_114_BE_5           in 114        , r^2=0.999512
PCM_Engine_Speed_RPM --> S_114_LE_54          in 114        , r^2=0.996725


In [127]:
s = decoder_FD3.get_signal('18DAF110', [5, 98, 1, 213], 'PCM_Engine_Speed_RPM')
print("----------------------\n"+f"{1}/{5:<10} Decoding:   {s.msg.msg_id:<8} | {s.name}")

----------------------
1/5          Decoding:   18DAF110 | PCM_Engine_Speed_RPM
